# Designing the JuliaCon 2019 T-shirt

## @Cormullion and David P. Sanders

@Cormullion was selected to design the JuliaCon 2019 T-shirt on the basis of his many contributions to design throughout the Julia ecosystem.

After several designs were proposed, a [Penrose tiling](https://en.wikipedia.org/wiki/Penrose_tiling) was chosen.

In this notebook we explain some of the steps in arriving at the final design.

The drawing was done using the [`Luxor.jl`](https://github.com/JuliaGraphics/Luxor.jl) package.

## Penrose tilings

A Penrose tiling is a tiling (covering) of the plane using two particular tile types:

[FIG of tile types]

The surprising thing is that the tiling turns out to be *quasiperiodic*: it never repeats itself, but does "almost repeat itself".

A common method to produce Penrose tilings is the method of **inflation&ndash;deflation**:

we start with a given set of tiles and recursively replace each tile with a certain combination of smaller tiles.

In this implementation, we build everything out of triangles, represented by the following `struct`. 
Here, `A`, `B` and `C` are the 3 points in the triangle, `Point` is a type from `Luxor` representing the coordinates of a point in 2D, and `flag` is a Boolean variable representing the "type" of triangle.

In [ ]:
using Luxor

struct Triangle
    flag::Bool 
    A::Point
    B::Point
    C::Point
end

We start by generating $n$ triangles in a circle with center $c$ and radius $r$. The `polar` function generates a vector with given radius and angle

In [ ]:
function initialize(center::Point, n)
    A = center

    for i in 1:n
        ϕ = (i - 1) * (2π / n)
        C = A + polar(r, ϕ)
        
        ϕ = (i) * (2π / n)
        B = A + polar(r, ϕ)
        
        if i % 2 == 1
            triangle = Triangle(true, B, A, C)
        else
            triangle = PenroseTriangle(true, C, A, B)
        end
        
        push!(triangles, triangle)
    end
    
    return triangles
end

Let's draw those triangles:

Now we subdivide recursively using the following function:

In [ ]:
function subdivide(triangles::Vector{Triangle})
    
    result = Triangle[]
    
    for triangle in triangles
        A, B, C = triangle.A, triangle.B, triangle.C

        if triangle.flag
            Q = A + (B - A) / MathConstants.golden
            R = B + (C - B) / MathConstants.golden
            
            push!(result, Triangle(false, R, Q, B))
            push!(result, Triangle(true,  Q, A, R))
            push!(result, Triangle(true,  C, A, R))

        else           
            P = C + (A - C) / MathConstants.golden
            
            push!(result, Triangle(false, B, P, A))
            push!(result, Triangle(true,  P, C, B))
        end
    end
    
    return result
end

Let's see the result of successive subdivisions (PREFERABLY SUPERIMPOSED SOMEHOW ON THE PREVIOUS ONE:)

## Colouring the triangles

A Penrose tiling can be thought of as a planar map, and the [Four-colour Theorem](https://en.wikipedia.org/wiki/Four_color_theorem) says that any map of the plane can be coloured using at most 4 colours. Here, a map colouring is one in which neighbouring "countries" are coloured by different colours if they are joined by an edge (not just a point).

Since there are 4 colours in the Julia logo, it seemed like a good idea to find a colouring of the Penrose tiling using the Julia colours! 

### Mapping to a graph

To do so, we think of the collection of triangles as a **graph** or **network**, i.e. a collection of vertices joined by edges. We need to find the **adjacency matrix** of the graph, i.e. a matrix $A_{ij}$ which contains $1$ if nodes $i$ and $j$ are connected, and $0$ if they are not connected.

The problem is that the inflation&ndash;deflation method that we are using to construct the triangles gives us no information about which triangles are neighbours (at least, we are not aware of how to directly extract this information from the construction).

Hence, the first thing to do is to extract the adjacency matrix, for which we just use brute force: given two triangles, we calculate the distances between each of their vertices. If exactly two of these distances are (approximately) zero, then those two vertices coincide, and hence the two triangles share an edge:

In [ ]:
using SimpleGraphs

function incidence_matrix(triangles::Vector{Triangle})

    n = length(triangles)

    g = IntGraph(n)

    for i in 1:n
        for j in i+1:n
            ti = triangles[i]
            tj = triangles[j]

            points_i = [ti.pointA, ti.pointB, ti.pointC]
            points_j = [tj.pointA, tj.pointB, tj.pointC]

            coincide = 0
            tol = 1e-2

            for k in 1:3, l in 1:3
                if distance(points_i[k], points_j[l]) < tol
                    coincide += 1
                end
            end

            if coincidental == 2  # share edge
                add!(g, i, j)
            end
        end
    end

    return g
end

FIGURE OF THE GRAPH SUPERIMPOSED ON THE TRIANGLES: JOIN THE CENTROID OF ADJACENT TRIANGLES WITH A LINE

### Colouring the graph

Finally, we want to find a **graph colouring** with the property that adjacent nodes do not share the same colour. This is accomplished by 

In [3]:
g = incidence_matrix(triangles)

colouring = greedy_color(g, [1:n;])

UndefVarError: UndefVarError: incidence_matrix not defined

It turns out that for the Penrose tiling, there are rather few nodes that use the fourth colour (i.e. that have 3 neighbouring nodes). FIGURE

To make the result visually more interesting, we randomly choose approximately a quarter of the nodes to have the 4th colour, provided that none of its neighbours have that colour:

In [ ]:
for i in 1:n ÷ 4
    which = rand(1:n)
    
    if 4 ∉ [coloring[x] for x in neighbors(g, which)]
        coloring[which] = 4
    end
fend